In [1]:
import datetime
import pytz
from simplenlg.features      import Feature, Tense, InternalFeature, NumberAgreement, Person
from simplenlg.framework     import DocumentElement, InflectedWordElement, LexicalCategory
from simplenlg.framework     import NLGElement, NLGFactory, StringElement, WordElement
from simplenlg.lexicon       import Lexicon, XMLLexicon
from simplenlg.phrasespec    import *
from simplenlg.realiser.english  import Realiser
from collections import OrderedDict
import inflect
import statistics
import random
import csv
import locale
import math
import datamuse

In [15]:
DataType = Enum('DataTypes', 'percent dollar number date string')
PeriodType = Enum('PeriodType', 'day week month quarter')


class Human():
    def get_positive(self):
        arr = ['yes', 'great', 'nice', 'awesome', 'good', 'wonderful']
        return arr[random.randint(0, len(arr)-1)].capitalize()

    def get_negative(self):
        arr = ['not good', 'oh no', 'i see', 'okay']
        return arr[random.randint(0, len(arr)-1)].capitalize()
    
    def get_neutral(self):
        arr = ['not bad!', 'Think about it!']
        return arr[random.randint(0, len(arr)-1)].capitalize()

    def get_greetings(self):
        arr = ['hey', 'hi', "how is it going",
               'how are you doing', "how is everything", 'good day']
        return arr[random.randint(0, len(arr)-1)].capitalize()

    def get_fill(self):
        arr = ['We found out', 'We noticed',
               'We observed', 'We discovered']
        return arr[random.randint(0, len(arr)-1)].capitalize()

    def get_discovered(self):
        arr = ['started looking at', 'looked into', 'looked at', 'dug into']
        return arr[random.randint(0, len(arr)-1)].capitalize()

    def get_period(self):
        arr = ['this week', 'this period',
               'so far this week', 'so far this period', '']
        return arr[random.randint(0, len(arr)-1)].capitalize()

    def get_lets_see(self):
        arr = ['']
        return arr[random.randint(0, len(arr)-1)].capitalize()

    def get_more(self):
        arr = ['Interested to know more?', 'Here are few more quick facts for you!...',
               'Still hungry for numbers?']
        return arr[random.randint(0, len(arr)-1)].capitalize()
    
    def get_lets_look_at(self):
        arr = ['Lets look at']
        return arr[random.randint(0, len(arr)-1)].capitalize()

    def get_direction(self, num):
        if(float(num) > 0):
            return "\'up\'"
        else:
            return "\'down\'"


class Period():

    def __init__(self, start_date, end_date, period_type=PeriodType.week):
        self.start_date = start_date
        self.end_date = end_date
        self.period_type = period_type

    def get_period_str(self):
        if PeriodType.day:
            return 'daily'
        elif PeriodType.week:
            return 'weekly'
        elif PeriodType.month:
            return 'monthly'
        elif PeriodType.quarter:
            return 'quarterly'

class StockDataDict():

    def __init__(self, symbol="symbol", last_week_close_price="", start_ts="", end_ts="", curr_period_avg="", 
                 curr_period_min="", curr_period_max="", curr_period_var="", numeric_units="", company="company", 
                 hq="HQ", domain="domain"):
        self.symbol = symbol
        self.last_week_close_price = last_week_close_price
        self.start_ts = start_ts
        self.end_ts = end_ts
        self.curr_period_avg = curr_period_avg
        self.curr_period_min = curr_period_min
        self.curr_period_max = curr_period_max
        self.curr_period_var = curr_period_var
        self.numeric_units = numeric_units
        self.company = company
        self.hq = hq
        self.domain = domain
        #self.period = "period" #calculate period based on start and end ts
        #self.key_units = key_units
        #self.measure = measure
        #self.measure_scale = measure_scale
        #self.convert_keys()
        #self.prev_data_dict = prev_data_dict
        #if prev_data_dict:
        #    self.compare_to_previous()

    def data_to_text_conversion_str(self, val):
        if(self.numeric_units == DataType.percent):
            return self.format_percent_str(val)
        elif (self.numeric_units == DataType.dollar):
            return self.convert_to_dollars_str(val)
        elif (self.numeric_units == DataType.number):
            return self.format_int_str(val)
        else:
            return val

    def data_to_text_conversion_val(self, val):
        return val

    def format_percent_str(self, percent_change, percentSign=True):
        return str(abs(round(percent_change, 2)))+"%"
    
    def format_int_str(self, val):
        return str(int(val))

    def convert_to_dollars_str(self, dollars):
        locale.setlocale(locale.LC_ALL, '')
        return locale.currency(dollars)


class Narrator():
    def __init__(self, period):
        self.period = period
        self.lexicon = XMLLexicon()
        self.nlgFactory = NLGFactory(self.lexicon)
        self.realiser = Realiser(self.lexicon)
        self.inflect_engine = inflect.engine()

    def format_percent(self, percent_change, percentSign=True):
        if(not percentSign):
            return "{0:.0}".format(percent_change)
        return "{0:.0%}".format(percent_change)

    def get_last(self, period_type):
        return {
            'day': 'yesterday',
            'week': 'last week',
            'month': 'last month',
            'quarter': 'last quarter',
        }[period_type]

    def reltime(self, date, compare_to=None, at='@'):
        r'''Takes a datetime and returns a relative representation of the
        time.
        :param date: The date to render relatively
        :param compare_to: what to compare the date to. Defaults to datetime.now()
        :param at: date/time separator. defaults to "@". "at" is also reasonable.
        >>> from datetime import datetime, timedelta
        >>> today = datetime(2050, 9, 2, 15, 00)
        >>> earlier = datetime(2050, 9, 2, 12)
        >>> reltime(earlier, today)
        'today @ 12pm'
        >>> yesterday = today - timedelta(1)
        >>> reltime(yesterday, compare_to=today)
        'yesterday @ 3pm'
        >>> reltime(datetime(2050, 9, 1, 15, 32), today)
        'yesterday @ 3:32pm'
        >>> reltime(datetime(2050, 8, 31, 16), today)
        'Wednesday @ 4pm (2 days ago)'
        >>> reltime(datetime(2050, 8, 26, 14), today)
        'last Friday @ 2pm (7 days ago)'
        >>> reltime(datetime(2049, 9, 2, 12, 00), today)
        'September 2nd, 2049 @ 12pm (last year)'
        >>> today = datetime(2012, 8, 29, 13, 52)
        >>> last_mon = datetime(2012, 8, 20, 15, 40, 55)
        >>> reltime(last_mon, today)
        'last Monday @ 3:40pm (9 days ago)'
        '''
        def ordinal(n):
            r'''Returns a string ordinal representation of a number
            Taken from: http://stackoverflow.com/a/739301/180718
            '''
            if 10 <= n % 100 < 20:
                return str(n) + 'th'
            else:
                return str(n) + {1 : 'st', 2 : 'nd', 3 : 'rd'}.get(n % 10, "th")

        compare_to = compare_to or datetime.now()
        if date > compare_to:
            return NotImplementedError('reltime only handles dates in the past')
        #get timediff values
        diff = compare_to - date
        if diff.seconds < 60 * 60 * 8: #less than a business day?
            days_ago = diff.days
        else:
            days_ago = diff.days + 1
        months_ago = compare_to.month - date.month
        years_ago = compare_to.year - date.year
        weeks_ago = int(math.ceil(days_ago / 7.0))
        #get a non-zero padded 12-hour hour
        hr = date.strftime('%I')
        if hr.startswith('0'):
            hr = hr[1:]
        wd = compare_to.weekday()
        #calculate the time string
        if date.minute == 0:
            time = '{0}{1}'.format(hr, date.strftime('%p').lower())
        else:
            time = '{0}:{1}'.format(hr, date.strftime('%M%p').lower())
        #calculate the date string
        if days_ago == 0:
            #datestr = 'today {at} {time}'
            datestr = 'today'
        elif days_ago == 1:
            #datestr = 'yesterday {at} {time}'
            datestr = 'yesterday'
        elif (wd in (5, 6) and days_ago in (wd+1, wd+2)) or \
                wd + 3 <= days_ago <= wd + 8:
            #this was determined by making a table of wd versus days_ago and
            #divining a relationship based on everyday speech. This is somewhat
            #subjective I guess!
            #datestr = 'last {weekday} {at} {time} ({days_ago} days ago)'
            datestr = 'last {weekday} {days_ago} days ago'
        elif days_ago <= wd + 2:
            #datestr = '{weekday} {at} {time} ({days_ago} days ago)'
            datestr = '{weekday} {days_ago} days ago'
        elif years_ago == 1:
            #datestr = '{month} {day}, {year} {at} {time} (last year)'
            datestr = '{month} last year'
        elif years_ago > 1:
            #datestr = '{month} {day}, {year} {at} {time} ({years_ago} years ago)'
            datestr = '{month} {years_ago} years ago'
        elif months_ago == 1:
            #datestr = '{month} {day} {at} {time} (last month)'
            datestr = 'last month'
        elif months_ago > 1:
            #datestr = '{month} {day} {at} {time} ({months_ago} months ago)'
            datestr = '{months_ago} months ago'
        else: 
            #not last week, but not last month either
            #datestr = '{month} {day} {at} {time} ({days_ago} days ago)'
            datestr = '{days_ago} days ago'
            
        #return datestr
        return datestr.format(time=time,
                              weekday=date.strftime('%A'),
                              day=ordinal(date.day),
                              days=diff.days,
                              days_ago=days_ago,
                              month=date.strftime('%B'),
                              years_ago=years_ago,
                              months_ago=months_ago,
                              weeks_ago=weeks_ago,
                              year=date.year,
                              at=at)
    
    def add_metric_chart(self, period_value, last_period_value, subject, value_title, numeric_units=DataType.number, period_overwrite=None, given_delta=None):
        segments = [{}]
        percent_change = 0
        try:
            percent_change = ((float(period_value)-float(last_period_value)) /
                              float(last_period_value)) if not given_delta else given_delta
        except ZeroDivisionError as e:
            pass
        direction_change = Human().get_direction(percent_change)
        period = self.period.period_type.name.lower(
        ) if not period_overwrite else period_overwrite
        direction_str = Human().get_direction(percent_change).replace('\'', '')
        percent_change_str = self.format_percent(abs(percent_change))
        period_value_str = StockDataDict(
            numeric_units=numeric_units).data_to_text_conversion_str(period_value)

        # https://github.com/bjascob/pySimpleNLG/blob/02db3ea1fc1d210c24c3046cd79384ede0fe8be9/tests/syntax/english/ClauseTest.py

        x_is_up = self.nlgFactory.createClause()
        x_is_up.setFeature(Feature.TENSE, Tense.PAST)
        x_is_up.setSubject(f"{value_title}")
        if not self.inflect_engine.singular_noun(value_title):
            x_is_up.setFeature(Feature.NUMBER, NumberAgreement.SINGULAR)
        else:
            x_is_up.setFeature(Feature.NUMBER, NumberAgreement.PLURAL)
        x_is_up.setVerb("be")
        x_is_up.setObject(self.nlgFactory.createNounPhrase(direction_str))
        x_is_up_str = self.realiser.realise(x_is_up).getRealisation()

        current_period = self.nlgFactory.createClause()
        current_period.setFeature(Feature.TENSE, Tense.PAST)
        current_period.setSubject(f"{value_title}")
        if not self.inflect_engine.singular_noun(value_title):
            current_period.setFeature(Feature.NUMBER, NumberAgreement.SINGULAR)
        else:
            current_period.setFeature(Feature.NUMBER, NumberAgreement.PLURAL)
        current_period.setVerb("be")
        current_period.setObject(str(period_value_str))
        current_period = self.realiser.realise(current_period).getRealisation()

        steps = [f"{subject}"]
        steps.append(
            f"{current_period} this {period}."
        )
        if(percent_change != 0):
            steps.append(
                f"{x_is_up_str} by {percent_change_str} from {self.get_last(period)}'s {str(last_period_value)}.")
        return steps

    def add_line_chart(self, entity1, entity2, subject, value_title, value1, value2, numeric_units=DataType.number):
        percent_change = 0
        try:
            percent_change = ((float(value1)-float(value2)) /
                              float(value2))
        except ZeroDivisionError as e:
            pass
        direction_change = Human().get_direction(percent_change)
        direction_str = Human().get_direction(percent_change).replace('\'', '')
        percent_change_str = self.format_percent(abs(percent_change))
        #period_value_str = DataDict(
        #    numeric_units=numeric_units).data_to_text_conversion_str(period_value)

        # https://github.com/bjascob/pySimpleNLG/blob/02db3ea1fc1d210c24c3046cd79384ede0fe8be9/tests/syntax/english/ClauseTest.py

        x_is_up = self.nlgFactory.createClause()
        x_is_up.setFeature(Feature.TENSE, Tense.PAST)
        x_is_up.setSubject(f"{entity1}'s {value_title}")
        if not self.inflect_engine.singular_noun(value_title):
            x_is_up.setFeature(Feature.NUMBER, NumberAgreement.SINGULAR)
        else:
            x_is_up.setFeature(Feature.NUMBER, NumberAgreement.PLURAL)
        x_is_up.setVerb("be")
        x_is_up.setObject(self.nlgFactory.createNounPhrase(direction_str))
        x_is_up_str = self.realiser.realise(x_is_up).getRealisation()
        
        steps = [f"{subject}"]
        api = datamuse.Datamuse()
        
        #dm_tried = [{'word':'tried'}]
        #dm_tried.extend(api.words(ml='tried', max=10))
        dm_tried = [{'word': 'tried'}, {'word': 'proved', 'score': 84677, 'tags': ['syn', 'adj']}, {'word': 'tested', 'score': 84208, 'tags': ['syn', 'adj']}, {'word': 'proven', 'score': 82470, 'tags': ['syn', 'adj']}, {'word': 'dependable', 'score': 78926, 'tags': ['syn', 'adj']}, {'word': 'reliable', 'score': 78581, 'tags': ['syn', 'adj']}, {'word': 'time-tested', 'score': 77429, 'tags': ['syn', 'adj']}, {'word': 'tried and true', 'score': 77429, 'tags': ['syn', 'adj']}, {'word': 'well-tried', 'score': 77429, 'tags': ['syn', 'adj']}, {'word': 'attempted', 'score': 77428, 'tags': ['adj', 'v']}, {'word': 'wanted', 'score': 71929, 'tags': ['adj', 'v']}]
        sel_tried = dm_tried[random.randint(0, len(dm_tried)-1)]

        #dm_comparative = [{'word':'comparative'}]
        #dm_comparative.extend(api.words(ml='comparative', max=10))
        dm_comparative = [{'word': 'comparative'}, {'word': 'relative', 'score': 83552, 'tags': ['syn', 'adj']}, {'word': 'comparison', 'score': 69209, 'tags': ['n']}, {'word': 'comparable', 'score': 68893, 'tags': ['adj']}, {'word': 'comparability', 'score': 67606, 'tags': ['n']}, {'word': 'comparing', 'score': 67268, 'tags': ['n']}, {'word': 'comparisons', 'score': 66205, 'tags': ['n']}, {'word': 'compare', 'score': 65949, 'tags': ['v']}, {'word': 'benchmarking', 'score': 65282, 'tags': ['adj']}, {'word': 'compared', 'score': 64795, 'tags': ['v']}, {'word': 'corresponding', 'score': 64091, 'tags': ['adj']}]
        sel_comparative = dm_comparative[random.randint(0, len(dm_comparative)-1)]

        #dm_study = [{'word':'study'}]
        #dm_study.extend(api.words(ml='study', max=10))
        dm_study = [{'word': 'study'}, {'word': 'survey', 'score': 96232, 'tags': ['syn', 'n']}, {'word': 'report', 'score': 89437, 'tags': ['syn', 'n']}, {'word': 'examine', 'score': 85312, 'tags': ['syn', 'v']}, {'word': 'work', 'score': 82653, 'tags': ['syn', 'n']}, {'word': 'analyze', 'score': 81788, 'tags': ['syn', 'v']}, {'word': 'meditate', 'score': 80487, 'tags': ['syn', 'v']}, {'word': 'learn', 'score': 79066, 'tags': ['syn', 'v']}, {'word': 'consider', 'score': 78968, 'tags': ['syn', 'v']}, {'word': 'analyse', 'score': 77657, 'tags': ['syn', 'v']}, {'word': 'sketch', 'score': 77583, 'tags': ['syn', 'n']}]
        sel_study = dm_study[random.randint(0, len(dm_study)-1)]
        
        steps.append(
            f"We {sel_tried['word']} to do a {sel_comparative['word']} {sel_study['word']} of {entity1} with {entity2}."
        )
        
        #dm_compared = [{'word':'compared'}]
        #dm_compared.extend(api.words(ml='compared', max=10))
        dm_compared = [{'word': 'compared'}, {'word': 'versus', 'score': 70844, 'tags': ['n', 'v']}, {'word': 'comparison', 'score': 70308, 'tags': ['n']}, {'word': 'contrasted', 'score': 68805, 'tags': ['v']}, {'word': 'equated', 'score': 66307, 'tags': ['v']}, {'word': 'comparative', 'score': 64868, 'tags': ['adj', 'n']}, {'word': 'matched', 'score': 62311, 'tags': ['adj', 'v']}, {'word': 'relative', 'score': 62259, 'tags': ['adj']}, {'word': 'comparisons', 'score': 61965, 'tags': ['n']}, {'word': 'measured', 'score': 61849, 'tags': ['adj', 'v']}, {'word': 'related', 'score': 61564, 'tags': ['adj', 'v']}]
        sel_compared = dm_compared[random.randint(0, len(dm_compared)-1)]
        
        if(percent_change != 0):
            steps.append(
                f"{x_is_up_str} by {percent_change} when {sel_compared['word']} to {entity2}'s {value_title}.")
        return steps
    
    #####
    def add_horzBar_chart(self, data_dict, title, period, prev_data_dict=None, diff_prev_entity=False):
        steps = []
        #steps = []
        avg_num = float(data_dict.curr_period_avg)
        
        #dm_organization = [{'word':'organization'}, {'word':'company'}]
        #dm_organization.extend(api.words(ml='organization', max=10))
        dm_organization = [{'word': 'organization'}, {'word': 'company'}, {'word': 'establishment', 'score': 80499, 'tags': ['syn', 'n']}, {'word': 'governance', 'score': 79453, 'tags': ['syn', 'n']}, {'word': 'system', 'score': 78919, 'tags': ['syn', 'n']}, {'word': 'formation', 'score': 77741, 'tags': ['syn', 'n']}, {'word': 'organisation', 'score': 76951, 'tags': ['syn', 'n']}, {'word': 'constitution', 'score': 76578, 'tags': ['syn', 'n']}, {'word': 'administration', 'score': 76073, 'tags': ['syn', 'n']}, {'word': 'arrangement', 'score': 75303, 'tags': ['syn', 'n']}, {'word': 'brass', 'score': 74898, 'tags': ['syn', 'n']}, {'word': 'group', 'score': 69572, 'tags': ['n']}]
        sel_organization = dm_organization[random.randint(0, len(dm_organization)-1)]
        #print(sel_organization)

        #dm_headquartered = [{'word':'headquartered'}]
        #dm_headquartered.extend(api.words(ml='headquartered', max=10))
        dm_headquartered=[{'word': 'headquartered'}, {'word': 'based', 'score': 70081, 'tags': ['adj', 'v']}, {'word': 'located', 'score': 69250, 'tags': ['adj', 'v']}, {'word': 'situated', 'score': 62356, 'tags': ['adj', 'v']}, {'word': 'established', 'score': 60773, 'tags': ['adj', 'v']}, {'word': 'stationed', 'score': 60609, 'tags': ['v']}, {'word': 'housed', 'score': 60320, 'tags': ['v']}, {'word': 'prepared', 'score': 53106, 'tags': ['adj']}, {'word': 'seat', 'score': 52692, 'tags': ['n']}, {'word': 'employs', 'score': 51143, 'tags': ['v']}, {'word': 'operates', 'score': 51142, 'tags': ['v']}]
        sel_headquartered = dm_headquartered[random.randint(0, len(dm_headquartered)-1)]
        
        steps.append(
            f"{data_dict.company} is a(n) {data_dict.domain} {sel_organization['word']} {sel_headquartered['word']} at {data_dict.hq}")
        
        dm_stockmarket = ['stock market', 'stock exchange']
        sel_stockmarket = dm_stockmarket[random.randint(0, len(dm_stockmarket)-1)]
        
        #dm_timeperiod = [{'word':'time period'}]
        #dm_timeperiod.extend(api.words(ml='time period', max=5))
        dm_timeperiod=[{'word': 'time period'}, {'word': 'period', 'score': 64621, 'tags': ['syn', 'n']}, {'word': 'period of time', 'score': 63576, 'tags': ['syn', 'n']}, {'word': 'era', 'score': 62529, 'tags': ['syn', 'n']}, {'word': 'day', 'score': 57329, 'tags': ['syn', 'n']}, {'word': 'eon', 'score': 57329, 'tags': ['syn', 'n']}]
        sel_timeperiod = dm_timeperiod[random.randint(0, len(dm_timeperiod)-1)]
        
        steps.append(
            f"{Human().get_lets_look_at()} the {data_dict.company}'s {sel_stockmarket} in the {sel_timeperiod['word']} from {data_dict.start_ts} to {data_dict.end_ts}")
        #steps.append(f"{Human().get_lets_see()}")
        
        #dm_highest = [{'word':'most'}, {'word':'highest'}, {'word':'greatest'}]
        #dm_highest.extend(api.words(ml='highest', max=15))
        dm_highest=[{'word': 'most'}, {'word': 'highest'}, {'word': 'greatest'}, {'word': 'maximum', 'score': 82033, 'tags': ['syn', 'adj']}, {'word': 'peak', 'score': 81553, 'tags': ['syn', 'adj', 'n']}, {'word': 'ultimate', 'score': 78440, 'tags': ['syn', 'adj']}, {'word': 'maximal', 'score': 76201, 'tags': ['syn', 'adj']}, {'word': 'strongest', 'score': 68244, 'tags': ['adj']}, {'word': 'loftiest', 'score': 67701, 'tags': ['adj']}, {'word': 'top', 'score': 67296, 'tags': ['adj']}, {'word': 'greatest', 'score': 67213, 'tags': ['adj']}, {'word': 'best', 'score': 66218, 'tags': ['adj', 'n']}, {'word': 'largest', 'score': 65954, 'tags': ['adj']}, {'word': 'widest', 'score': 65694, 'tags': ['adj']}, {'word': 'worst', 'score': 65648, 'tags': ['adj']}, {'word': 'topmost', 'score': 65563, 'tags': ['adj']}, {'word': 'biggest', 'score': 65459, 'tags': ['adj']}, {'word': 'fastest', 'score': 64957, 'tags': ['adv', 'adj']}]
        sel_highest = dm_highest[random.randint(0, len(dm_highest)-1)]
        
        #dm_lowest = [{'word':'lowest'}]
        #dm_lowest.extend(api.words(ml='lowest', max=15))
        dm_lowest=[{'word': 'lowest'}, {'word': 'worst', 'score': 88319, 'tags': ['syn', 'adj']}, {'word': 'smallest', 'score': 87730, 'tags': ['syn', 'adj']}, {'word': 'minimum', 'score': 81602, 'tags': ['syn', 'adj', 'n']}, {'word': 'last', 'score': 81060, 'tags': ['syn', 'adj']}, {'word': 'least', 'score': 79667, 'tags': ['syn', 'adj']}, {'word': 'bottom', 'score': 78608, 'tags': ['syn', 'adj']}, {'word': 'minimal', 'score': 77839, 'tags': ['syn', 'adj']}, {'word': 'last-place', 'score': 71197, 'tags': ['syn', 'adj']}, {'word': 'weakest', 'score': 71196, 'tags': ['adj']}, {'word': 'fewest', 'score': 70085, 'tags': ['adj']}, {'word': 'slowest', 'score': 66344, 'tags': ['adv', 'adj']}, {'word': 'cheapest', 'score': 66234, 'tags': ['adj']}, {'word': 'shortest', 'score': 65088, 'tags': ['adj']}, {'word': 'below', 'score': 64758, 'tags': ['adv']}, {'word': 'best', 'score': 64732, 'tags': ['adj']}]
        sel_lowest = dm_lowest[random.randint(0, len(dm_lowest)-1)]
        
        dm_price = ['price', 'value', 'cost', 'quote']
        sel_price = dm_price[random.randint(0, len(dm_price)-1)]
        
        #dm_approximately = [{'word':'approximately'}]
        #dm_approximately.extend(api.words(ml='approximately', max=15))
        dm_approximately=[{'word': 'approximately'}, {'word': 'roughly', 'score': 94205, 'tags': ['syn', 'adv']}, {'word': 'around', 'score': 82887, 'tags': ['syn', 'adv']}, {'word': 'about', 'score': 80101, 'tags': ['syn', 'adv']}, {'word': 'more or less', 'score': 78759, 'tags': ['syn', 'adv']}, {'word': 'some', 'score': 78120, 'tags': ['syn', 'adv']}, {'word': 'close to', 'score': 75489, 'tags': ['syn', 'adv']}, {'word': 'just about', 'score': 73011, 'tags': ['syn', 'adv']}, {'word': 'or so', 'score': 72686, 'tags': ['syn', 'adv']}, {'word': 'approx', 'score': 72213, 'tags': ['n']}, {'word': 'estimated', 'score': 69496, 'tags': ['adj', 'v']}, {'word': 'nearly', 'score': 66875, 'tags': ['adv']}, {'word': 'average', 'score': 64748, 'tags': ['adj', 'n', 'adv', 'v']}, {'word': 'substantially', 'score': 61718, 'tags': ['adv']}, {'word': 'significantly', 'score': 60474, 'tags': ['adv']}, {'word': 'almost', 'score': 60151, 'tags': ['adv']}]
        sel_approximately = dm_approximately[random.randint(0, len(dm_approximately)-1)]
            
        #dm_compared = [{'word':'compared'}]
        #dm_compared.extend(api.words(ml='compared', max=10))
        dm_compared=[{'word': 'compared'}, {'word': 'versus', 'score': 70844, 'tags': ['n', 'v']}, {'word': 'comparison', 'score': 70308, 'tags': ['n']}, {'word': 'contrasted', 'score': 68805, 'tags': ['v']}, {'word': 'equated', 'score': 66307, 'tags': ['v']}, {'word': 'comparative', 'score': 64868, 'tags': ['adj', 'n']}, {'word': 'matched', 'score': 62311, 'tags': ['adj', 'v']}, {'word': 'relative', 'score': 62259, 'tags': ['adj']}, {'word': 'comparisons', 'score': 61965, 'tags': ['n']}, {'word': 'measured', 'score': 61849, 'tags': ['adj', 'v']}, {'word': 'related', 'score': 61564, 'tags': ['adj', 'v']}]
        sel_compared = dm_compared[random.randint(0, len(dm_compared)-1)]

        steps.append(
            f"{data_dict.company} had the {sel_highest['word']} stock {sel_price} with ${data_dict.curr_period_max}")
        steps.append(
            f"{data_dict.company} had the {sel_lowest['word']} stock price with ${data_dict.curr_period_min}")
        steps.append(f"And the average this {sel_timeperiod['word']} is {sel_approximately['word']} ${data_dict.curr_period_avg}, give or take {data_dict.curr_period_var}")
        
        today = datetime.datetime.now()
        relperiod = self.reltime(today, today)
        #print(relperiod)
        period = "week"
        if prev_data_dict != None:
            if data_dict.symbol == prev_data_dict.symbol:
                diff_prev_entity = False
                date1 = datetime.datetime.strptime(prev_data_dict.start_ts, '%d_%b_%Y')
                date2 = datetime.datetime.strptime(data_dict.start_ts, '%d_%b_%Y')
                if date1 > date2:
                    temp = date2
                    date2 = date1
                    date1 = temp
                relperiod = self.reltime(date1, date2)
                print(relperiod)
                if "day" in relperiod:
                    period = "day"
                elif "week" in relperiod:
                    period = 'week'
                elif "month" in relperiod:
                    period = "month"
                elif "year" in relperiod:
                    period = "year"
            else:
                diff_prev_entity = True
        

        #dm_increase = [{'word':'increase'}]
        #dm_increase.extend(api.words(ml='increase', max=15))
        dm_increase=[{'word': 'increase'}, {'word': 'growth', 'score': 120808, 'tags': ['syn', 'n']}, {'word': 'gain', 'score': 119119, 'tags': ['syn', 'v', 'n']}, {'word': 'increment', 'score': 118913, 'tags': ['syn', 'n']}, {'word': 'addition', 'score': 112300, 'tags': ['syn', 'n']}, {'word': 'step-up', 'score': 105114, 'tags': ['syn', 'n']}, {'word': 'decrease', 'score': 105113, 'tags': ['n', 'v', 'ant']}, {'word': 'rise', 'score': 98407, 'tags': ['n', 'v']}, {'word': 'decline', 'score': 97684, 'tags': ['n', 'v']}, {'word': 'spike', 'score': 96069, 'tags': ['n']}, {'word': 'upsurge', 'score': 95837, 'tags': ['n']}, {'word': 'doubling', 'score': 95187, 'tags': ['n']}, {'word': 'higher', 'score': 94979, 'tags': ['adj']}, {'word': 'improvement', 'score': 94451, 'tags': ['n']}, {'word': 'boost', 'score': 94202, 'tags': ['n', 'v']}, {'word': 'escalation', 'score': 93683, 'tags': ['n']}]
        sel_increase = dm_increase[random.randint(0, len(dm_increase)-1)]
        
        #dm_decrease = [{'word':'decrease'}]
        #dm_decrease.extend(api.words(ml='decrease', max=15))
        dm_decrease=[{'word': 'decrease'}, {'word': 'reduction', 'score': 128795, 'tags': ['syn', 'n']}, {'word': 'diminution', 'score': 121587, 'tags': ['syn', 'n']}, {'word': 'lessening', 'score': 120654, 'tags': ['syn', 'n']}, {'word': 'lessen', 'score': 118908, 'tags': ['syn', 'v', 'n']}, {'word': 'diminish', 'score': 117613, 'tags': ['syn', 'v']}, {'word': 'decrement', 'score': 116952, 'tags': ['syn', 'n']}, {'word': 'fall', 'score': 115989, 'tags': ['syn', 'v', 'n']}, {'word': 'drop-off', 'score': 105118, 'tags': ['syn', 'n']}, {'word': 'step-down', 'score': 105118, 'tags': ['syn', 'n']}, {'word': 'increase', 'score': 105117, 'tags': ['n', 'v', 'adj', 'ant']}, {'word': 'decline', 'score': 100253, 'tags': ['n', 'v']}, {'word': 'reductions', 'score': 96896, 'tags': ['n']}, {'word': 'reduce', 'score': 96065, 'tags': ['v', 'n']}]
        sel_decrease = dm_decrease[random.randint(0, len(dm_decrease)-1)]
        
        dm_last = ['last', 'past', 'previous', 'latest', 'latter']
        sel_last = dm_last[random.randint(0, len(dm_last)-1)]
        
        #dm_next = [{'word':'next'}]
        #dm_next.extend(api.words(ml='next', max=15))
        dm_next=[{'word': 'next'}, {'word': 'future', 'score': 82102, 'tags': ['syn', 'adj']}, {'word': 'following', 'score': 78347, 'tags': ['syn', 'adj', 'v']}, {'word': 'close', 'score': 74475, 'tags': ['syn', 'adj']}, {'word': 'succeeding', 'score': 74381, 'tags': ['syn', 'v']}, {'word': 'incoming', 'score': 74045, 'tags': ['syn', 'adj']}, {'word': 'adjacent', 'score': 70880, 'tags': ['syn', 'adj']}, {'word': 'side by side', 'score': 68622, 'tags': ['syn', 'adj']}, {'word': 'last', 'score': 68621, 'tags': ['adj']}, {'word': 'later', 'score': 67766, 'tags': ['adv']}, {'word': 'first', 'score': 66709, 'tags': ['adj', 'n']}, {'word': 'upcoming', 'score': 66681, 'tags': ['adj']}, {'word': 'coming', 'score': 66523, 'tags': ['n', 'v']}, {'word': 'ahead', 'score': 65899, 'tags': ['adv']}, {'word': 'expected', 'score': 64567, 'tags': ['adj']}, {'word': 'second', 'score': 64101, 'tags': ['adj', 'n']}]
        sel_next = dm_next[random.randint(0, len(dm_next)-1)]
        
        #dm_expected = [{'word':'expected'}, {'word':'projected'}]
        #dm_expected.extend(api.words(ml='expected', max=15))
        dm_expected=[{'word': 'expected'}, {'word': 'projected'}, {'word': 'likely', 'score': 102293, 'tags': ['syn', 'adj']}, {'word': 'anticipated', 'score': 101205, 'tags': ['syn', 'adj', 'v']}, {'word': 'predicted', 'score': 99744, 'tags': ['syn', 'adj', 'v']}, {'word': 'supposed', 'score': 94975, 'tags': ['syn', 'adj', 'v']}, {'word': 'due', 'score': 94220, 'tags': ['syn', 'adj']}, {'word': 'awaited', 'score': 92717, 'tags': ['syn', 'v']}, {'word': 'foreseen', 'score': 90287, 'tags': ['syn', 'adj']}, {'word': 'potential', 'score': 86611, 'tags': ['syn', 'adj', 'n']}, {'word': 'foretold', 'score': 86041, 'tags': ['syn', 'adj']}, {'word': 'unsurprising', 'score': 86016, 'tags': ['syn', 'adj']}, {'word': 'prospective', 'score': 84239, 'tags': ['syn', 'adj']}, {'word': 'hoped-for', 'score': 79977, 'tags': ['syn', 'adj']}, {'word': 'matter-of-course', 'score': 79977, 'tags': ['syn', 'adj']}, {'word': 'slated', 'score': 78513, 'tags': ['v']}, {'word': 'scheduled', 'score': 77463, 'tags': ['adj', 'v']}]
        sel_expected = dm_expected[random.randint(0, len(dm_expected)-1)]
        
        #dm_trend = [{'word':'trend'}]
        #dm_trend.extend(api.words(ml='trend', max=15))
        dm_trend=[{'word': 'trend'}, {'word': 'vogue', 'score': 86524, 'tags': ['syn', 'n']}, {'word': 'tendency', 'score': 85031, 'tags': ['syn', 'n']}, {'word': 'drift', 'score': 84072, 'tags': ['syn', 'n']}, {'word': 'curve', 'score': 83434, 'tags': ['syn', 'n']}, {'word': 'style', 'score': 80197, 'tags': ['syn', 'n']}, {'word': 'slew', 'score': 79128, 'tags': ['syn', 'n']}, {'word': 'veer', 'score': 79058, 'tags': ['syn', 'v']}, {'word': 'course', 'score': 78914, 'tags': ['syn', 'n']}, {'word': 'slue', 'score': 78455, 'tags': ['syn', 'v']}, {'word': 'swerve', 'score': 77678, 'tags': ['syn', 'n']}, {'word': 'cut', 'score': 75482, 'tags': ['syn', 'n']}, {'word': 'sheer', 'score': 74189, 'tags': ['syn', 'adj']}, {'word': 'pattern', 'score': 70179, 'tags': ['n']}, {'word': 'downtrend', 'score': 69353, 'tags': ['n']}, {'word': 'phenomenon', 'score': 68249, 'tags': ['n']}]
        sel_trend = dm_trend[random.randint(0, len(dm_trend)-1)]
        
        #dm_understand = [{'word':'understand'}, {'word':'make sense out of'}, {'word':'read'}]
        #dm_understand.extend(api.words(ml='understand', max=15))
        dm_understand=[{'word': 'understand'}, {'word': 'make sense out of'}, {'word': 'read'}, {'word': 'realize', 'score': 98620, 'tags': ['syn', 'v']}, {'word': 'empathize', 'score': 94176, 'tags': ['syn', 'v']}, {'word': 'sympathize', 'score': 93347, 'tags': ['syn', 'v']}, {'word': 'interpret', 'score': 93237, 'tags': ['syn', 'v']}, {'word': 'infer', 'score': 91127, 'tags': ['syn', 'v']}, {'word': 'see', 'score': 90852, 'tags': ['syn', 'v']}, {'word': 'read', 'score': 88372, 'tags': ['syn', 'v']}, {'word': 'translate', 'score': 86458, 'tags': ['syn', 'v']}, {'word': 'gather', 'score': 84482, 'tags': ['syn', 'v']}, {'word': 'comprehend', 'score': 75670, 'tags': ['v']}, {'word': 'know', 'score': 74661, 'tags': ['v', 'n']}, {'word': 'explain', 'score': 74490, 'tags': ['v']}, {'word': 'recognize', 'score': 71816, 'tags': ['v']}, {'word': 'appreciate', 'score': 71293, 'tags': ['v']}, {'word': 'learn', 'score': 70076, 'tags': ['v']}]
        sel_understand = dm_understand[random.randint(0, len(dm_understand)-1)]
        
        
        if prev_data_dict != None and not diff_prev_entity:
            #steps.append(
            #    f"{Human().get_fill()} the {title} with past {data_dict.period}")
            #past_avg_num = prev_data_dict.get_average()
            past_avg_value = prev_data_dict.curr_period_avg
            past_avg_num = float(prev_data_dict.curr_period_avg)
            past_var_value = prev_data_dict.curr_period_var
            steps.append(f"{relperiod}, the average stock {sel_price} for {data_dict.company} was ${past_avg_value}, give or take {past_var_value}")
            if (avg_num > past_avg_num):
                diff = data_dict.data_to_text_conversion_str(avg_num - past_avg_num)
                projection = data_dict.data_to_text_conversion_str(avg_num + (avg_num - past_avg_num))
                steps.append(f"That is an average {sel_increase['word']} of ${diff} {sel_compared['word']} to {sel_last} {period}! {Human().get_positive()}")
                steps.append(f"The {sel_expected['word']} average stock {sel_price} for {data_dict.company} in the {sel_next['word']} {period} is {projection}!")
            if (avg_num < past_avg_num):
                diff = data_dict.data_to_text_conversion_str(past_avg_num - avg_num)
                projection = data_dict.data_to_text_conversion_str(avg_num - (past_avg_num - avg_num))
                steps.append(f"That is an average {sel_decrease['word']} of ${diff} {sel_compared['word']} to {sel_last} {period}! {Human().get_negative()}")
                steps.append(f"The {sel_expected['word']} average stock {sel_price} for {data_dict.company} in the {sel_next['word']} {period} is {projection}!")
            if (avg_num == past_avg_num):
                steps.append(f"The stock {sel_price} average is keeping up at {past_avg_value} {sel_compared['word']} to {sel_last} {period}! {Human().get_neutral()}")
            metrics_title = f"{Human().get_more()}. Here is some Averages {sel_trend['word']} to {sel_understand['word']}!"
            metrics_subject = f"{data_dict.company} Averages"
            metrics = self.add_metric_chart(avg_num, past_avg_num, metrics_title, metrics_subject)
            for step in metrics:
                steps.append(step)
                                                                                          
        #dm_sector = [{'word':'sector'}]
        #dm_sector.extend(api.words(ml='sector', max=15))
        dm_sector=[{'word': 'sector'}, {'word': 'sphere', 'score': 90711, 'tags': ['syn', 'n']}, {'word': 'industries', 'score': 72882, 'tags': ['n']}, {'word': 'industry', 'score': 71815, 'tags': ['n']}, {'word': 'economy', 'score': 69727, 'tags': ['n']}, {'word': 'companies', 'score': 69230, 'tags': ['n']}, {'word': 'market', 'score': 68603, 'tags': ['n']}, {'word': 'sectoral', 'score': 68418, 'tags': ['adj']}, {'word': 'sectorial', 'score': 66538, 'tags': ['adj']}, {'word': 'government', 'score': 65086, 'tags': ['n']}, {'word': 'enterprises', 'score': 64985, 'tags': ['n']}, {'word': 'industrial', 'score': 64793, 'tags': ['adj', 'n']}, {'word': 'institutions', 'score': 64647, 'tags': ['n']}, {'word': 'agriculture', 'score': 64546, 'tags': ['n']}, {'word': 'manufacturing', 'score': 63992, 'tags': ['n', 'v']}, {'word': 'businesses', 'score': 63792, 'tags': ['n']}]
        sel_sector = dm_sector[random.randint(0, len(dm_sector)-1)]        
        if prev_data_dict != None and diff_prev_entity:
            #steps.append(
            #    f"{Human().get_fill()} the {title} of {data_dict.business_units} with {prev_data_dict.business_units}")
            steps.append(
                f"{prev_data_dict.company} is a(n) {sel_organization['word']} in {prev_data_dict.domain} {sel_sector['word']} {sel_headquartered['word']} at {prev_data_dict.hq}")
            past_avg_value = prev_data_dict.curr_period_avg
            past_avg_num = float(prev_data_dict.curr_period_avg)
            past_var_value = prev_data_dict.curr_period_var
            steps.append(f"{prev_data_dict.company} averages at ${past_avg_value}, give or take {past_var_value}")
            if (avg_num > past_avg_num):
                diff = data_dict.data_to_text_conversion_str(avg_num - past_avg_num)
                projection = data_dict.data_to_text_conversion_str(avg_num + (avg_num - past_avg_num))
                steps.append(f"That is an average {sel_increase['word']} of {diff} for {prev_data_dict.company} {sel_compared['word']} to {data_dict.company} stock {sel_price}! {Human().get_positive()}")
                #steps.append(f"The projected average {data_dict.measure} next {data_dict.period} is {projection}!")
            if (avg_num < past_avg_num):
                diff = data_dict.data_to_text_conversion_str(past_avg_num - avg_num)
                projection = data_dict.data_to_text_conversion_str(avg_num - (past_avg_num - avg_num))
                steps.append(f"That is an average {sel_decrease['word']} of {diff} for {prev_data_dict.company} {sel_compared['word']} to {data_dict.company} stock {sel_price}! {Human().get_negative()}")
                #steps.append(f"The projected average {data_dict.measure} next {data_dict.period} is {projection}!")
            if (avg_num == past_avg_num):
                steps.append(f"{sel_stockmarket} averages for both {data_dict.company} and {prev_data_dict.company} has been trending at {past_avg_value} for the {sel_last} few {period}s! {Human().get_neutral()}")
            metrics_title = f"{Human().get_more()}"
            metrics = self.add_line_chart(data_dict.company, prev_data_dict.company, metrics_title, "Averages", avg_num, past_avg_num)
            for step in metrics:
                steps.append(step)
                
        return steps

In [16]:
from random import sample

def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    s1 = sample(s, 4) #len(s)
    # traverse in the string   
    for ele in s1:  
        str1 += ele + '.'
    
    # return string   
    return str1  

In [17]:
import csv
import json

f = open("summary.txt", "a")
f1 = open("input.txt", "a")

with open('export.csv', mode='r') as infile:
    reader = csv.reader(infile)
    for row in reader:
        for i, _ in enumerate(row):
            row[i] = row[i].replace(", ", ",")
            row[i] = row[i].replace(' ', '_')
        mydict_obj = StockDataDict(
        symbol=row[0],
        last_week_close_price='0',
        start_ts = datetime.datetime.strptime(row[9],"%d_%b_%Y"),
        end_ts= (datetime.datetime.strptime(row[9],"%d_%b_%Y") + datetime.timedelta(days = (7))).strftime("%d_%b_%Y"),
        curr_period_avg=row[1],
        curr_period_min=row[2],
        curr_period_max=row[3],
        curr_period_var=row[4],
        numeric_units=DataType.dollar,
        company=row[5],
        hq=row[8],
        domain=row[6] + '(' + row[7] + ')'
        )
        summary = (Narrator('week').add_horzBar_chart(mydict_obj, "", '1 Week'))
        f.write(listToString(summary) + '\n')
        input_line = "symbol_1:"+row[0] + " 	startts_1:" + row[9] + " 	endts_1:" + (datetime.datetime.strptime(row[9] , "%d_%b_%Y")  +  datetime.timedelta(days = (7))).strftime("%d_%b_%Y") + " 	currperiodavg_1:"+row[1] + " 	currperiodmin_1:" + row[2] + " 	currperiodmax_1:" + row[3] + " 	currperiodvar_1:" + row[4] + " 	numericunits_1:$" + " 	company_1:" + row[5] + " 	hq_1:" + row[8] + " 	domain_1:" + row[6]  + " 	domain_2:" +  '('  +  " 	domain_3:" + row[7]  + " 	domain_4:" + ')'
        f1.write(input_line + '\n')
        print(row)
        
f.close()
f1.close()

['MSFT', '60.52', '59.87', '61', '0.27', 'Microsoft', 'Information_Technology', 'Systems_Software', 'Redmond,Washington', '28_Oct_2016']
['MSFT', '57.67', '57.44', '58.12', '0.07', 'Microsoft', 'Information_Technology', 'Systems_Software', 'Redmond,Washington', '19_Aug_2016']
['MSFT', '57.94', '57.67', '58.17', '0.03', 'Microsoft', 'Information_Technology', 'Systems_Software', 'Redmond,Washington', '26_Aug_2016']
['AAPL', '109.71', '107.79', '111.06', '2.25', 'Apple', 'Information_Technology', 'Computer_Hardware', 'Cupertino,California', '11_Nov_2016']
['MSFT', '59.76', '58.7', '60.47', '0.69', 'Microsoft', 'Information_Technology', 'Systems_Software', 'Redmond,Washington', '11_Nov_2016']
['AAPL', '108.56', '105.71', '110.06', '4.11', 'Apple', 'Information_Technology', 'Computer_Hardware', 'Cupertino,California', '18_Nov_2016']
['MSFT', '62.54', '62.17', '62.98', '0.1', 'Microsoft', 'Information_Technology', 'Systems_Software', 'Redmond,Washington', '16_Dec_2016']
['MSFT', '62.83', '62

In [18]:
import csv
import json

f = open("summary.txt", "a")
f1 = open("input.txt", "a")
start_date = datetime.datetime.strptime('2016-08-19', '%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2016-12-31', '%Y-%m-%d').date()
period = Period(start_date, end_date, PeriodType.quarter)
stock_narrator = Narrator(period)

with open('export.csv', mode='r') as infile:
    reader = csv.reader(infile)
    for row in reader:
        for i, _ in enumerate(row):
            row[i] = row[i].replace(",", " ")
            row[i] = row[i].replace(' ', '_')
        mydict_obj = StockDataDict(
        symbol=row[0],
        last_week_close_price='0',
        start_ts = row[9],
        end_ts= (datetime.datetime.strptime(row[9],"%d_%b_%Y") + datetime.timedelta(days = (7))).strftime("%d_%b_%Y"),
        curr_period_avg=row[1],
        curr_period_min=row[2],
        curr_period_max=row[3],
        curr_period_var=row[4],
        numeric_units=DataType.dollar,
        company=row[5],
        hq=row[8],
        domain=row[6] + '(' + row[7] + ')'
        )
        with open('export.csv', mode='r') as infile1:
            reader1 = csv.reader(infile1)
            for row1 in reader1:
                for i, _ in enumerate(row1):
                    row1[i] = row1[i].replace(",", " ")
                    row1[i] = row1[i].replace(' ', '_')
                mydict_obj1 = StockDataDict(
                symbol=row1[0],
                last_week_close_price='0',
                start_ts = row1[9],
                end_ts= (datetime.datetime.strptime(row1[9],"%d_%b_%Y") + datetime.timedelta(days = (7))).strftime("%d_%b_%Y"),
                curr_period_avg=row1[1],
                curr_period_min=row1[2],
                curr_period_max=row1[3],
                curr_period_var=row1[4],
                numeric_units=DataType.dollar,
                company=row1[5],
                hq=row1[8],
                domain=row1[6] + '(' + row1[7] + ')'
                )
                summary = (stock_narrator.add_horzBar_chart(mydict_obj, "", '1 Week', mydict_obj1))
                f.write(listToString(summary) + '\n')
                input_line = "symbol_1:"+row[0] + " 	startts_1:" + row[9] + " 	endts_1:" + (datetime.datetime.strptime(row[9] , "%d_%b_%Y")  +  datetime.timedelta(days = (7))).strftime("%d_%b_%Y") + " 	currperiodavg_1:"+row[1] + " 	currperiodmin_1:" + row[2] + " 	currperiodmax_1:" + row[3] + " 	currperiodvar_1:" + row[4] + " 	numericunits_1:$" + " 	company_1:" + row[5] + " 	hq_1:" + row[8] + " 	domain_1:" + row[6]  + " 	domain_2:" +  '('  +  " 	domain_3:" + row[7]  + " 	domain_4:" + ')'+" 	symbol_2:"+row1[0] + " 	startts_2:" + row1[9] + " 	endts_2:" + (datetime.datetime.strptime(row1[9] , "%d_%b_%Y")  +  datetime.timedelta(days = (7))).strftime("%d_%b_%Y") + " 	currperiodavg_2:"+row1[1] + " 	currperiodmin_2:" + row1[2] + " 	currperiodmax_2:" + row1[3] + " 	currperiodvar_2:" + row1[4] + " 	numericunits_2:$" + " 	company_2:" + row1[5] + " 	hq_2:" + row1[8] + " 	domain_1:" + row[6]  + " 	domain_2:" +  '('  +  " 	domain_3:" + row[7]  + " 	domain_4:" + ')'
                f1.write(input_line + '\n')
        
f.close()
f1.close()

today
2 months ago
2 months ago
last month
2 months ago
2 months ago
last Friday 7 days ago
2 months ago
last month
last month
last Friday 7 days ago
last month
last month
last month
last month
21 days ago
14 days ago
2 months ago
last month
2 months ago
2 months ago
today
last Friday 7 days ago
3 months ago
4 months ago
4 months ago
3 months ago
4 months ago
3 months ago
last month
2 months ago
last month
3 months ago
last month
last month
2 months ago
2 months ago
4 months ago
last month
4 months ago
2 months ago
last Friday 7 days ago
today
3 months ago
4 months ago
4 months ago
3 months ago
4 months ago
3 months ago
last month
2 months ago
last month
3 months ago
last Friday 7 days ago
last month
2 months ago
2 months ago
4 months ago
last month
4 months ago
today
last Friday 7 days ago
14 days ago
last month
last month
3 months ago
2 months ago
last month
2 months ago
3 months ago
2 months ago
last Friday 7 days ago
last month
last month
last month
2 months ago
last month
last mon